# Módulo 5 Tarefa 1
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [1]:
!pip install --upgrade bottleneck --user

In [2]:
import pandas as pd
import requests

# 1) seu código aqui
df = pd.read_csv('SINASC_RO_2019.csv')
df.drop_duplicates().shape
# Não há duplicados

(27028, 69)

In [3]:
# 2) seu código aqui
df.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

In [4]:
# 3) seu código aqui
df = df[['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']]
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [5]:
# 4) seu código aqui
df.dropna(subset=['APGAR5'], inplace=True)
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [6]:
# 5) seu código aqui
df[df[['ESTCIVMAE', 'CONSULTAS']].isna()] = 9
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [7]:
# 6) Seu código aquiaqui
df[df[['QTDFILVIVO']].isna()] = 0
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [8]:
# 7) Valores não preenchidos de escolaridade, gravidez e gestação serão considerados ignorados (código 9)
df[df[['ESCMAE', 'GESTACAO', 'GRAVIDEZ']].isna()] = 9
df.isna().sum()

LOCNASC       0
IDADEMAE      0
ESTCIVMAE     0
ESCMAE        0
QTDFILVIVO    0
GESTACAO      0
GRAVIDEZ      0
CONSULTAS     0
APGAR5        0
dtype: int64

In [18]:
# 8)     
df['Classificação_APGAR'] = ''  # Create a new column with empty values

# Assign values based on conditions
df.loc[df['apgar5'] <= 3, 'Classificação_APGAR'] = 'asfixia severa'
df.loc[(df['apgar5'] > 3) & (df['apgar5'] <= 5), 'Classificação_APGAR'] = 'asfixia moderada'
df.loc[(df['apgar5'] > 5) & (df['apgar5'] <= 7), 'Classificação_APGAR'] = 'asfixia leve'
df.loc[df['apgar5'] > 7, 'Classificação_APGAR'] = 'normal'


df[df['apgar5'] < 7]

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,classificação_apgar,Classificação_APGAR,Classificação_APGAR5
20,4,29,5.0,4 a 7 anos,3.0,32 a 36 semanas,Única,3,0.0,asfixia severa,asfixia severa,
182,1,30,5.0,8 a 11 anos,2.0,22 a 27 semanas,Única,3,4.0,asfixia moderada,asfixia moderada,
185,1,24,1.0,8 a 11 anos,0.0,22 a 27 semanas,Única,3,3.0,asfixia severa,asfixia severa,
205,1,28,2.0,4 a 7 anos,2.0,37 a 41 semanas,Única,3,6.0,asfixia leve,asfixia leve,
289,1,25,5.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,6.0,asfixia leve,asfixia leve,
...,...,...,...,...,...,...,...,...,...,...,...,...
26893,1,21,5.0,1 a 3 anos,4.0,37 a 41 semanas,Única,2,6.0,asfixia leve,asfixia leve,
26915,1,36,5.0,4 a 7 anos,5.0,37 a 41 semanas,Única,3,5.0,asfixia moderada,asfixia moderada,
26921,1,20,1.0,4 a 7 anos,1.0,37 a 41 semanas,Única,3,6.0,asfixia leve,asfixia leve,
26932,1,18,2.0,4 a 7 anos,0.0,32 a 36 semanas,Única,4,0.0,asfixia severa,asfixia severa,


In [27]:
# 9) Renomeie as variáveis para que fiquem no snake case, ou seja, em letras minúsculas, com um underscore entre as palávras. 
#Dica: repare que se você não quiser criar um dataframe novo, você vai precisar usar a opção inplace = True.
import re

def snake_case(string):
    # Replace spaces and punctuation with underscores
    string = re.sub(r'[\s\W_]+', '_', string)
    # Convert to lowercase
    string = string.lower()
    # Remove leading and trailing underscores
    string = string.strip('_')
    return string

df.columns = df.columns.map(snake_case)
df

,locnasc,idademae,estcivmae,escmae,qtdfilvivo,gestacao,gravidez,consultas,apgar5,classificação_apgar,classificação_apgar,classificação_apgar5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,normal,
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal,normal,
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal,normal,
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal,normal,
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal,normal,
...,...,...,...,...,...,...,...,...,...,...,...,...
27023,1,32,2.0,12 anos ou mais,1.0,32 a 36 semanas,Única,4,9.0,normal,normal,
27024,1,19,1.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,9.0,normal,normal,
27025,1,24,2.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal,normal,
27026,1,21,2.0,8 a 11 anos,1.0,32 a 36 semanas,Única,4,9.0,normal,normal,


In [29]:
# reproduzir os agrupamentos realizados com o groupby utilizando o pivot_table().

pivot_df = pd.pivot_table(df, values='classificação_apgar', aggfunc='sum')

print(pivot_df)


ValueError: No group keys passed!